In [1]:
!pip install psycopg2
!pip install openpyxl

In [51]:
import requests
from pathlib import Path
import json
import pandas as pd
import datetime
from datetime import datetime, timezone
import hashlib
from sqlalchemy import create_engine
import numpy as np
from time import sleep
import logging
import pytz
import math
import os

In [52]:
from dotenv import load_dotenv
env_file = Path("../.env")
print(".env exists:", env_file.exists(), "→", env_file)
# Загружаем .env
loaded = load_dotenv(dotenv_path=env_file)

.env exists: True → ..\.env


In [32]:
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.width', 1000)  # Увеличить ширину отображения

In [31]:
# Текущее время в московском часовом поясе
moscow_tz = pytz.timezone('Europe/Moscow')
current_time = datetime.now(moscow_tz).strftime('%Y-%m-%dT%H:%M:%S.%f%z')
# Уубираем двоеточие
current_time = current_time[:-2] + ':' + current_time[-2:]

In [30]:
url = os.getenv('URL')
headers = {
    'Content-Type': os.getenv('CONTENT_TYPE'),
    'Authorization': os.getenv('AUTHORIZATION'),
}

In [53]:
payload = {
    "otype": "WarehouseStates",
    "com": "execOperation",
    "op": "static/getTotals()",
    "opargs": {
"theCard": {
"dateTime": current_time,
"groupBy": "pat"
}
}
}


response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
#print(response.text)
goods_data = response.json()
df_wr = pd.DataFrame(goods_data["resData"]["rows"])

200


In [54]:
display(df_wr)

,id,batchId,batchCode,patId,patName,patNumGosReg,warehouseId,warehouseName,contractorId,contractorName,contractorInn,countNetto,countPu,puUnit,note,validFrom,puKgWeight
0,33148797,463001168588,13123123123,3922,Удобрение комплексное минеральное с микроэлеме...,345-13-3801-1,1077174,"(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)",248824,"ООО ""ЛЕ МОНЛИД""",5029069967,60,12.0000000000000000,Упаковка,инвентаризация. Ввод остатков,2025-04-14T21:00:00.001Z,5
1,33148800,463001168591,13123123123,1000,"2,4-Дактив",063-03-2788-1,1077174,"(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)",248824,"ООО ""ЛЕ МОНЛИД""",5029069967,60,12.0000000000000000,Банка,инвентаризация. Ввод остатков,2025-04-10T21:00:00.001Z,5
2,33148885,463001168626,9876,3163,Карбамид марка: Б,321-10-763-1,1077174,"(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)",248824,"ООО ""ЛЕ МОНЛИД""",5029069967,11,11.0000000000000000,Упаковка,инвентаризация. Ввод остатков,2025-04-14T14:13:53.826Z,1
3,33148886,463001168627,5423,5538,Фертика бесхлорное калийное удобрение с микроэ...,841-17-3868-1,1077174,"(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)",248824,"ООО ""ЛЕ МОНЛИД""",5029069967,11,11.0000000000000000,Упаковка,инвентаризация. Ввод остатков,2025-04-14T14:13:53.826Z,1
4,33149005,463001168673,6789,3770,Кристалон марки: Кристалон Желтый 13+40+13; Кр...,186-13-3428-1,1077174,"(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)",248824,"ООО ""ЛЕ МОНЛИД""",5029069967,11,11.0000000000000000,Упаковка,инвентаризация. Ввод остатков,2025-04-14T14:43:42.379Z,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,33300994,463001168568,13123123123,1000,"2,4-Дактив",063-03-2788-1,1077174,"(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)",248824,"ООО ""ЛЕ МОНЛИД""",5029069967,0.000000,0E-20,Банка,списание партии - отправка на розничную продажу,2025-12-02T11:19:46.604666Z,5
68,33301049,463001167054,121212,1000,"2,4-Дактив",063-03-2788-1,1062746,"(RU-077/EA84820) ООО ""Леруа Мерлен Восток"" / Т...",248824,"ООО ""ЛЕ МОНЛИД""",5029069967,0.000000,0E-20,Бак,списание партии - отправка на розничную продажу,2025-12-03T18:12:41.287010Z,5
69,33301050,463001168560,13123,1000,"2,4-Дактив",063-03-2788-1,1077174,"(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)",248824,"ООО ""ЛЕ МОНЛИД""",5029069967,0.000000,0E-20,Банка,списание партии - отправка на розничную продажу,2025-12-03T18:12:41.287010Z,5
70,33148883,463001168624,1445,3106,"Аммофос высший сорт, первый сорт",321-10-4023-1,1077174,"(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)",248824,"ООО ""ЛЕ МОНЛИД""",5029069967,11,11.0000000000000000,Упаковка,инвентаризация. Ввод остатков,2025-04-14T14:13:53.826Z,1


In [35]:
goods_data

{'resCode': '200',
 'resMsg': 'Ok',
 'resData': {'rows': [{'id': '33148797',
    'batchId': '463001168588',
    'batchCode': '13123123123',
    'patId': '3922',
    'patName': 'Удобрение комплексное минеральное с микроэлементами (тукосмесь) марки: Фертика Универсал-2; Фертика Картофельное-5; Фертика Полевое; Фертика Осеннее; Фертика Свекловичное-6; Фертика Газонное. Весна-Лето; Фертика Газонное. Осень; Фертика Цветочное; Фертика Плюс',
    'patNumGosReg': '345-13-3801-1',
    'warehouseId': '1077174',
    'warehouseName': '(RU-077/RD83952) 86_ТЦ "Леруа Мерлен" (Троицк)',
    'contractorId': '248824',
    'contractorName': 'ООО "ЛЕ МОНЛИД"',
    'contractorInn': '5029069967',
    'countNetto': '60',
    'countPu': '12.0000000000000000',
    'puUnit': 'Упаковка',
    'note': 'инвентаризация. Ввод остатков',
    'validFrom': '2025-04-14T21:00:00.001Z',
    'puKgWeight': '5'},
   {'id': '33148800',
    'batchId': '463001168591',
    'batchCode': '13123123123',
    'patId': '1000',
    'pat

In [46]:
items_per_invoice=3
max_invoices=3

In [47]:
def prepare_invoices(df, items_per_invoice, max_invoices):
    """
    Формирует накладные на основе принципа FIFO (по validFrom) 
    
    Параметры:
    - df: загружаемый датафрейм с текущими остатками по складам
    - items_per_invoice: ограничение по максимальному количеству позиций в одной накладной
    - max_invoices: ограничение по максимальному количество накладных (если None - все возможные)
    """
    
    # 1. Подготовка данных
    # Преобразуем validFrom в datetime для сортировки
    df['validFrom'] = pd.to_datetime(df['validFrom'],
                                     utc = True,
                                     errors = 'coerce')
    df['countPu'] = pd.to_numeric(df['countPu'], errors='coerce')
    
    # Сортируем по дате validFrom (FIFO) и batchId для группировки
    df_sorted = df.sort_values(['validFrom', 'batchId']).reset_index(drop=True)
    
    # 2. Группируем по batchId, складываем countPu
    batch_groups = df_sorted.groupby(['batchId', 'patId','warehouseId']).agg({
        'countPu': 'sum',
        'validFrom': 'min',  # Берем самую раннюю дату
        'contractorId': 'first'
    }).reset_index()
    
    # 3. Формируем накладные
    invoices = []
    current_invoice_items = []
    current_total_count = 0
    
    for _, row in batch_groups.iterrows():
        batch_count_pu = row['countPu']
        
        # Если количество в партии больше, чем помещается в текущую накладную
        while batch_count_pu > 0:
            # Сколько можно добавить в текущую накладную
            available_space = items_per_invoice - len(current_invoice_items)
            
            if available_space > 0:
                # Добавляем всю партию или часть
                current_invoice_items.append({
                    'batchId': row['batchId'],
                    'countPuSent': float(batch_count_pu)
                })
                current_total_count += 1
                
                # Если накладная заполнилась или партия закончилась
                if len(current_invoice_items) >= items_per_invoice or batch_count_pu <= 0:
                    invoices.append(create_invoice_payload(current_invoice_items, row))
                    current_invoice_items = []
                    current_total_count = 0
                
                batch_count_pu = 0  # Вся партия добавлена
            else:
                # Текущая накладная заполнена, создаем новую
                invoices.append(create_invoice_payload(current_invoice_items, row))
                current_invoice_items = []
                current_total_count = 0
    
    # Добавляем последнюю неполную накладную, если есть
    if current_invoice_items:
        invoices.append(create_invoice_payload(current_invoice_items, batch_groups.iloc[-1]))
    
    # Ограничиваем количество накладных если нужно
    if max_invoices:
        invoices = invoices[:max_invoices]
    
    return invoices


def create_invoice_payload(items, row):
    """Создает payload для одной накладной"""
    current_time = datetime.now().isoformat() + 'Z'
    
    # Формируем номер накладной 
    doc_num = f"{datetime.now().strftime('%Y%m%d')}-{len(items):03d}"
    
    return {
        "com": "execOperation",
        "oid": "0",
        "op": "static/createNew()",
        "opargs": {
            "theCard": {
                "head": {
                    "docDate": current_time,
                    "docNote": f"Invoice_to_retail_{len(items)}_items",
                    "docNum": doc_num,
                    "name": f"Подготовка для списания в розницу ({len(items)} позиций)",
                    "receiverContractorId": int(row['contractorId']),
                    "sourceWarehouseId": int(row['warehouseId'])
                },
                "tbrDtoList": [
                    {
                        "batchId": int(item['batchId']),
                        "countPuSent": item['countPuSent']
                    } for item in items
                ]
            }
        },
        "otype": "Invoice"
    }


def create_retail_payload(invoice_id):
    """Создает payload для отправки накладной в розницу"""
    return {
        "com": "execOperation",
        "oid": str(invoice_id),
        "op": "draft/doSendToRetale()",
        "opargs": {},
        "otype": "Invoice"
    }


# Основной алгоритм выполнения
def process_invoices_to_retail(df, items_per_invoice=5, max_invoices=None, execute=True):
    """
    Полный процесс: формирование и отправка накладных
    
    Параметры:
    - df: исходный датафрейм
    - items_per_invoice: позиций в накладной
    - max_invoices: лимит накладных
    - execute: если False - только формирование, без отправки
    """
    
    # 1. Формируем накладные
    invoices = prepare_invoices(df, items_per_invoice, max_invoices)
    
    print(f"Сформировано {len(invoices)} накладных:")
    for i, invoice in enumerate(invoices, 1):
        items_count = len(invoice['opargs']['theCard']['tbrDtoList'])
        print(f"  Накладная {i}: {items_count} позиций")
    
    # 2. Отправляем накладные 
    results = []
    
    if execute:
        for i, invoice_payload in enumerate(invoices, 1):
            try:
               
                response = requests.post(url, headers=HEADERS, json=invoice_payload)

                draft_invoice_id = response.json()
                
                invoice_id = draft_invoice_id['resData']['id']

                retail_payload = create_retail_payload(invoice_id)
                                       
                # Отправляем в розницу

                print("Отправляем на розницу")
                print(retail_payload)
                
                retail_response = requests.post(url, headers=HEADERS, json=retail_payload)

                print("HTTP status:", response.status_code)
                    
                results.append({
                    'invoice_num': invoice_payload['opargs']['theCard']['head']['docNum'],
                    'invoice_id': invoice_id,
                    'items_count': len(invoice_payload['opargs']['theCard']['tbrDtoList']),
                    'status': 'created_and_sent'
                })
                
                print(f"✓ Накладная {i} (ID: {invoice_id}) отправлена в розницу")
                
            except Exception as e:
                results.append({
                    'invoice_num': invoice_payload['opargs']['theCard']['head']['docNum'],
                    'error': str(e),
                    'status': 'error'
                })
                print(f"✗ Ошибка с накладной {i}: {e}")
    
    return invoices, results

In [48]:
print("ОТПРАВКА НАКЛАДНЫХ:")
print("-" * 50)

# Отправка (execute=True)
invoices, results = process_invoices_to_retail(
    df=df_wr,
    items_per_invoice=items_per_invoice,
    max_invoices=max_invoices,
    execute=True  
)

# Результаты
print(f"\nИТОГИ:")
successful = len([r for r in results if r.get('status') == 'created_and_sent'])
failed = len([r for r in results if r.get('status') == 'error'])
print(f"Успешно отправлено: {successful}")
print(f"С ошибками: {failed}")

ОТПРАВКА НАКЛАДНЫХ:
--------------------------------------------------
Сформировано 3 накладных:
  Накладная 1: 3 позиций
  Накладная 2: 3 позиций
  Накладная 3: 3 позиций
✗ Ошибка с накладной 1: 'id'
✗ Ошибка с накладной 2: 'id'
Отправляем на розницу
{'com': 'execOperation', 'oid': '4842848', 'op': 'draft/doSendToRetale()', 'opargs': {}, 'otype': 'Invoice'}
HTTP status: 200
✓ Накладная 3 (ID: 4842848) отправлена в розницу

ИТОГИ:
Успешно отправлено: 1
С ошибками: 2


In [49]:
invoices

[{'com': 'execOperation',
  'oid': '0',
  'op': 'static/createNew()',
  'opargs': {'theCard': {'head': {'docDate': '2025-12-04T14:48:36.288551Z',
     'docNote': 'Invoice_to_retail_3_items',
     'docNum': '20251204-003',
     'name': 'Подготовка для списания в розницу (3 позиций)',
     'receiverContractorId': 248824,
     'sourceWarehouseId': 1077174},
    'tbrDtoList': [{'batchId': 463001168584, 'countPuSent': 8.0},
     {'batchId': 463001168588, 'countPuSent': 12.0},
     {'batchId': 463001168589, 'countPuSent': 12.0}]}},
  'otype': 'Invoice'},
 {'com': 'execOperation',
  'oid': '0',
  'op': 'static/createNew()',
  'opargs': {'theCard': {'head': {'docDate': '2025-12-04T14:48:36.288551Z',
     'docNote': 'Invoice_to_retail_3_items',
     'docNum': '20251204-003',
     'name': 'Подготовка для списания в розницу (3 позиций)',
     'receiverContractorId': 248824,
     'sourceWarehouseId': 1077174},
    'tbrDtoList': [{'batchId': 463001168590, 'countPuSent': 10.0},
     {'batchId': 4630